# Talks markdown generator for academicpages

Takes a TSV of talks with metadata and converts them for use with [xju2.github.io](xju2.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `talks.py`. Run either from the `markdown_generator` folder after replacing `talks.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases, rather than Stuart's non-standard TSV format and citation style.

In [2]:
import pandas as pd
import os

## Data format

The TSV needs to have the following columns: title, type, url_slug, venue, date, location, talk_url, description, with a header at the top. Many of these fields can be blank, but the columns must be in the TSV.

- Fields that cannot be blank: `title`, `url_slug`, `date`. All else can be blank. `type` defaults to "Talk" 
- `date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. 
    - The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/talks/YYYY-MM-DD-[url_slug]`
    - The combination of `url_slug` and `date` must be unique, as it will be the basis for your filenames

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [3]:
!cat talks.tsv

title	type	url_slug	venue	date	location	talk_url	description
Background estimation in four-lepton analysis   Talk    talk-1  Laboratoire de l'Accelerateur Lineaire-Orsay    2012-10-10  Orsay, France   I presented something    
Talk 1 on Relevant Topic in Your Field	Talk	talk-1	UC San Francisco, Department of Testing	2012-03-01	San Francisco, California		This is a description of your talk, which is a markdown files that can be all markdown-ified like any other post. Yay markdown!
Tutorial 1 on Relevant Topic in Your Field	Tutorial	tutorial-1	UC-Berkeley Institute for Testing Science	2013-03-01	Berkeley CA, USA	http://exampleurl.com	This is a description of your tutorial, note the different field in type. This is a markdown files that can be all markdown-ified like any other post. Yay markdown!
Talk 2 on Relevant Topic in Your Field	Talk	talk-2	London School of Testing	2014-02-01	London, UK	http://example2.com	This is a description of your talk, which is a markdown files that can be all 

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [32]:
talks = pd.read_csv("talks.tsv", sep="\t", header=0)
talks

,title,type,url_slug,venue,date,location,talk_url,description
0,Background estimation in four-lepton analysis,Talk@HZZ workshop,t,Laboratoire de l'Accelerateur Lineaire-Orsay,2012-10-10,"Orsay, France",NaN,I presented the estimation of reducible backgr...
1,Observation of the new Higgs-like particle in ...,Poster@Conference,p,CERN,2013-03-13,"Geneva, Switzerland",https://cds.cern.ch/record/1522394/files/ATL-C...,It describes the observation of a Higgs-like p...
2,2D workspace for mass and rate measurement,Talk@HZZ workshop,t,Sapienza Universita di Roma,2013-04-17,"Rome, Italy",https://indico.cern.ch/event/243532/contributi...,I presented the implementation of a two-dimens...
3,Observation and coupling measurements of the H...,Talk@US ATLAS General Meeting,t,Argonne National Laboratory,2013-07-17,"Lemont, IL, US",NaN,The measured Higgs coupling to fermions and ve...
4,Observation and measurements of the Higgs boso...,Talk@International Symposium on Higgs Physics,t,IHEP,2013-08-15,"Beijing, China",http://indico.ihep.ac.cn/event/3029/session/5/...,I presented the observation of the Higgs boson...
5,Selected key perspectives of H &rarr; ZZ analy...,Talk@Higgs workshop,t,Sapienza Universita di Roma,2014-04-18,"Rome, Italy",https://indico.cern.ch/event/289241/contributi...,LHC was expected to start its second run of pr...
6,Search for BSM physics including dark matter a...,Talk@2016 UCLA Dark Matter,t,University of California Los Angeles,2016-02-18,"Los Angeles, USA",https://conferences.pa.ucla.edu/dm16/talks/xia...,I presented the search for beyond the Standard...
7,Statistical framework in H&rarr;ZZ,Talk@HZZ workshop,t,Max-Planck-Institut fur Physik,2016-04-26,"Munich, German",https://indico.cern.ch/event/481860/contributi...,NaN
8,Study of the properties of the Higgs boson and...,Seminar@LPHE,s,Ecole Polytechnique Federale de Lausanne,2016-10-03,"Lausanne, Switzerland",http://lphe.epfl.ch/seminar/extern/Xiangyang_J...,I presented the latest measurements of the Hig...
9,Recent results of the Higgs measurements and s...,Seminar@Nanjing University,s,Nanjing University,2016-12-21,"Nanjing, China",NaN,I presented the recent results on the Higgs ph...


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [15]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    if type(text) is str:
        return "".join(html_escape_table.get(c,c) for c in text)
    else:
        return "False"

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [19]:
loc_dict = {}
idx_row = 0
for row, item in talks.iterrows():
    idx_row += 1
    # md_filename = str(item.date) + "-" + item.url_slug + ".md"
    # html_filename = str(item.date) + "-" + item.url_slug 
    md_filename = str(item.date) + "-" + item.url_slug + str(idx_row) + ".md"
    html_filename = str(item.date) + "-" + item.url_slug + str(idx_row)
    year = item.date[:4]
    
    md = "---\ntitle: \""   + item.title + '"\n'
    md += "collection: talks" + "\n"
    
    if len(str(item.type)) > 3:
        md += 'type: "' + item.type + '"\n'
    else:
        md += 'type: "Talk"\n'
    
    md += "permalink: /talks/" + html_filename + "\n"
    
    if len(str(item.venue)) > 3:
        md += 'venue: "' + item.venue + '"\n'
        
    if len(str(item.location)) > 3:
        md += "date: " + str(item.date) + "\n"
    
    if len(str(item.location)) > 3:
        md += 'location: "' + str(item.location) + '"\n'
           
    md += "---\n"
    
    
    if len(str(item.talk_url)) > 3:
        md += "\n[More information here](" + item.talk_url + ")\n" 
        
    
    if len(str(item.description)) > 3:
        md += "\n" + html_escape(item.description) + "\n"
        
        
    md_filename = os.path.basename(md_filename)
    #print(md)
    
    with open("../_talks/" + md_filename, 'w') as f:
        f.write(md)

These files are in the talks directory, one directory below where we're working from.

In [20]:
!ls ../_talks

2012-10-10-t1.md 2013-04-17-t3.md 2013-08-15-t5.md 2016-02-18-t7.md
2013-03-13-p2.md 2013-07-17-t4.md 2014-04-18-t6.md


In [7]:
!cat ../_talks/2013-03-01-tutorial-1.md

---
title: "Tutorial 1 on Relevant Topic in Your Field"
collection: talks
type: "Tutorial"
permalink: /talks/2013-03-01-tutorial-1
venue: "UC-Berkeley Institute for Testing Science"
date: 2013-03-01
location: "Berkeley CA, USA"
---

[More information here](http://exampleurl.com)

This is a description of your tutorial, note the different field in type. This is a markdown files that can be all markdown-ified like any other post. Yay markdown!
